In [1]:
import pandas as pd

In [2]:
# change settings to account for massive possible number if image_ids per play
pd.set_option('max_colwidth',1000)
pd.set_option('max_seq_items','none')

In [3]:
# read csvs as data frames
# cast as string to avoid float/NaN errors

plays = pd.read_csv('in/plays.csv', dtype=object)
play_auth = pd.read_csv('in/authors_plays.csv', dtype=object)
play_char = pd.read_csv('in/characters_plays.csv', dtype=object)
play_img = pd.read_csv('in/plays_images.csv', dtype=object)
play_perf = pd.read_csv('in/performances.csv', dtype=object)
imagesonline = pd.read_csv('in/imagesonline.csv', dtype=object)
play_scene = pd.read_csv('in/plays_scenes.csv', dtype=object)
play_kashira = pd.read_csv('in/kashira_plays.csv', dtype=object)

In [5]:
# mild clean up
play_perf = play_perf.drop('data_id', 1).drop('code', 1).drop('production_id',1)
imagesonline = imagesonline.rename(columns = {'id':'image_id'})

In [6]:
plays.head()

,play_id,label_ja,label_ja_sort,label_ka,sort_ja,label_eng,label_eng_sort,first_staged,reference
0,1,Onnakoroshi abura no jigoku,Onnakoroshi abura no jigoku,女殺油地獄,おんなころしあぶらのじごく,The Woman-Killer and the Hell of Oil,"Woman-Killer and the Hell of Oil, The",1721,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku February 1982 Performance Program"
1,2,Shinjū ten no Amijima,Shinju ten no Amijima,心中天網島,しんじゅうてんのあみじま,The Love Suicides at Amijima,"Love Suicides at Amijima, The",1720,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku October 1986 Performance Program"
2,3,Sonezaki shinjū,Sonezaki shinju,曽根崎心中,そねざきしんじゅう,The Love Suicides at Sonezaki,"Love Suicides at Sonezaki, The",1703,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Keene, Donald, Bunraku: The Art of the Japanese Puppet Theatre (Tokyo: Kodansha International, 1973); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku February 1984, February 1981 Performance Programs"
3,4,Shinjū Yoigōshin,Shinju Yoigoshin,心中宵庚申,しんじゅうよいごうしん,The Love Suicide of Hambei and Ochiyo,"Love Suicide of Hambei and Ochiyo, The",1722,"Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku May 1975, February 1982 Performance Program; Chikamatsu: 5 Late Plays, translated and annotated by C. Andrew Gerstle (New York: Columbia University Press, c2001)"
4,5,Somemoyō imose no kadomatsu,Somemoyo imose no kadomatsu,染模様妹背門松,そめもよういもせのかげまつ,The Love of Osome and Hisamatsu,"Love of Osome and Hisamatsu, The",1767,"Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Bunraku May 1976, November 1984 Performance Programs"


In [7]:
# clean out non-online images
imagesonline = imagesonline[['image_id']]
play_img = pd.merge(play_img, imagesonline, on='image_id', how='inner')

In [8]:
play_img.head()

,play_id,image_id
0,26,8281
1,26,8282
2,26,8283
3,26,8284
4,26,8285


In [9]:
# multiValDataFrame takes a dataframe df, column name df_index and column name multival_key
# it merges records with the same df_index and updates multival_index to an array of all the multival_index values.
# E.g for a data frame plays with multiple character_id values per play_id, returns a data frame with one record 
# per play_id, and one array of character_ids per play.

def multiValDataFrame(df, df_index, multival_key):
    new_df = pd.DataFrame({})
    cols = df.columns
    id_list = df[df_index].unique()
    for i in id_list:
        temp_row = df.loc[df[df_index] == i ]
        vals = []
        for col in cols:
            vals.append(temp_row[:1][col].values[0])

        temp_dict = dict(zip(cols, vals))
        multi_id_list = list(temp_row[multival_key])
    
        temp_dict[multival_key] = multi_id_list
    
        new_df = new_df.append(temp_dict, ignore_index=True)
    return new_df

In [10]:
# merge and multival authors
plays = pd.merge(plays, play_auth, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'author_id')

In [11]:
# merge and multival characters
plays = pd.merge(plays, play_char, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'character_id')

In [12]:
# merge and multival images
plays = pd.merge(plays, play_img, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'image_id')

In [13]:
# merge and multival performances
plays = pd.merge(plays, play_perf, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'performance_id')

In [14]:
# merge and multival scenes
plays = pd.merge(plays, play_scene, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'scene_id')

In [15]:
# merge and multival kashira
plays = pd.merge(plays, play_kashira, on='play_id', how='left')
plays = multiValDataFrame(plays, 'play_id', 'kashira_id')

In [16]:
plays = plays.rename(columns = {'play_id':'id'})
plays = plays[['id','author_id','character_id','scene_id','performance_id','kashira_id','first_staged','image_id','label_eng','label_ja','label_ka','reference']]

In [18]:
# export to csv
plays.to_csv('plays_exp.csv', encoding='utf8', index=False)

In [19]:
# export to json
plays.to_json('plays_exp.json', orient="records", force_ascii=False)

In [17]:
plays.head()

,id,author_id,character_id,scene_id,performance_id,kashira_id,first_staged,image_id,label_eng,label_ja,label_ka,reference
0,1,[6],"[1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 2298, 2307, 2317, 2319]","[309, 310, 311, 309, 310, 311, 309, 310, 311, 309, 310, 311, 312, 309, 310, 311, 312, 309, 310, 311, 309, 310, 311, 312, 309, 310, 311, 309, 310, 311, 312, 309, 310, 311]","[99, 194, 287, 414, 556, 576, 651, 778, 881, 976]","[15, 103, 33, 111, 103, 77, 116, 87, 54, 118, 66, 90, 111, 35, 13, 94, 70, 15, 103, 33, 111, 103, 77, 116, 87, 54, 118, 66, 90, 111, 35, 13, 94, 70, 15, 103, 33, 111, 103, 77, 116, 87, 54, 118, 66, 90, 111, 35, 13, 94, 70, 42, 77, 33, 87, 111, 103, 116, 118, 54, 35, 66, 35, 13, 90, 111, 33, 42, 35, 33, 70, 66, 90, 15, 77, 33, 87, 111, 103, 116, 118, 54, 35, 66, 35, 13, 90, 111, 33, 42, 35, 33, 70, 66, 90, 15, 9, 33, 87, 94, 111, 103, 116, 118, 54, 35, 66, 35, 89, 90, 111, 33, 15, 35, 33, 70, 66, 90, 15, 35, 15, 33, 35, 54, 15, 9, 33, 87, 87, 111, 103, 116, 118, 54, 35, 66, 35, 94, 90, 111, 33, 15, 35, 33, 70, 66, 90, 15, 35, 15, 33, 35, 54, 15, 111, 33, 87, 54, 35, 103, 116, 118, 66, 35, 89, 90, 111, 15, 35, 33, 70, 66, 90, 15, 33, 87, 87, 111, 103, 87, 118, 53, 35, 66, 35, 94, 90, 111, 33, 15, 35, 33, 70, 66, 90, 15, 35, 15, 33, 35, 53, 15, 87, 33, 87, 87, 111, 103, 116, 118, 53, 35, 66, 35, 94, 90, 111, 33, 15, 35, 33, 70, 66, 90, 15, 15, 94, 33, 87, 87, 111, 103, 87, 118, 53, 35...",1721,"[10777, 10778, 10779, 10780, 10781, 10782, 10783, 10784, 10785, 10786, 10787, 10788, 10789, 10892, 10893, 10894, 10895, 10896, 10897, 10898, 10899, 10900, 10901, 10902, 10903, 10904, 10905, 10906, 10907, 10908, 10909, 10910, 10911, 10912, 10915, 10916, 10917, 10918, 10919, 10920, 10921, 10922, 10923, 10924, 10925, 10926, 10927, 10928, 10929, 10930, 10931, 10932, 10933, 10934, 10935, 10936, 10937, 10938, 10939, 10940, 10941, 10942, 10943, 10944, 10945, 10946, 10947, 10948, 10949, 10950, 10951, 10952, 10953, 10954, 10955, 10956, 10957, 10958, 10959, 10960, 10961, 10962, 10963, 10964, 10965, 10966, 10967, 10968, 10980, 11356, 11357, 11358, 11359, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11376, 11377, 11378, 11379, 11380, 11381, 11382, 11383, 11384, 11385, 11386, 11387, 45061, 45089, 45095, 45100, 45101, 46082, 46084, 46086, 46092, 46250, 46251, 46252, 46253, 46254, 46255, 46256, 46282, 46283, 46284, 46285, 46286, 4...",The Woman-Killer and the Hell of Oil,Onnakoroshi abura no jigoku,女殺油地獄,"Major Plays of Chikamatsu, translated by Donald Keene, (New York: Columbia University Press, 1961); Hironaga, Shūzaburō, The Bunraku Handbook (Tokyo: Maison des Arts, 1976); Kokuritsu Gekijō jōen shiryōshū, edited by Kokuritsu Gekijō Geinō Chōsashitsu, vol. 224, February 1984; Bunraku February 1982 Performance Program"
1,2,[6],"[1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1701, 1878, 1879, 1880, 2146, 2147]","[330, 331, 532, 330, 331, 332, 532, 531, 330, 331, 332, 330, 331, 332, 531, 330, 331, 532, 330, 331, 332, 330, 331, 332, 532, 330, 331, 332, 532, 330, 331, 332, 532, 330, 331, 332, 330, 331, 332, 532, 330, 331, 330, 331, 332, 532]","[110, 262, 315, 383, 457, 554, 558, 573, 780, 791, 913, 951, 954]","[36, 111, 94, 70, 48, 103, 70, 87, 94, 36, 92, 15, 80, 53, 111, 6, 48, 111, 111, 103, 70, 79, 36, 14, 33, 94, 103, 6, 15, 80, 111, 36, 90, 53, 14, 33, 94, 36, 6, 80, 111, 111, 103, 111, 70, 79, 36, 111, 33, 94, 103, 111, 70, 79, 36, 111, 33, 94, 103, 6, 80, 14, 15, 90, 36, 54, 111, 33, 94, 36, 6, 80, 111, 36, 111, 94, 70, 79, 103, 6, 80, 14, 15, 90, 36, 54, 111, 33, 94, 36, 6, 80, 111, 103, 111, 70, 79, 36, 111, 33, 94, 103, 111, 70, 79, 36, 111, 33, 94, 103, 70, 87, 94, 36, 90, 15, 80, 54, 111, 6, 79, 111, 36, 111, 94, 70, 79, 103, 6, 80, 14, 15, 90, 36, 54, 111, 33, 94, 36, 6, 80, 111, 36, 111, 94, 70, 79, 103, 14, 15, 6, 36, 90, 53, 80, 